<center><img width="60%" src="https://github.com/jovianosilveira/img/blob/master/joviano-silveira-cotacoes-moedas-acoes-python-google-finance.png?raw=true"></center>

# **Cotações de moedas e ações**
*by [JOVIANO SILVEIRA](https://www.joviano.com)*


Vamos então carregar dados com Python carregando os dados do Google Finanças (através do Google Sheets), do Yahoo Finanças (através do pandas_datareader), e de quebra, a cotação histórica do Dolar através de API do Governo Brasileiro.



Obter preços históricos de ações  nunca foi tão fácil, rápido e barato.

Existem API's no mercado, usado por day traders, para consulta de dados em REAL TIME, mas, escrevi 3 métodos para obtenção de cotações históricas para nós pobres mortais, que querem analisar o passado, sem passar por aquele diário frio na barriga.

Ah, o intuito desse artigo é te mostrar como capturar os dados. 

Gerar os gráficos é assunto para um próximo artigo.

Antes de começar, foque em tudo, não só na obtenção de dados...ou seja, veja como aplicamos conceitos basicos de programação com Python, usando laços, dicionarios entre vários outros macetes.

## Bibliotecas usadas no projeto

In [1]:
# importando bibliotecas necessárias
import pandas as pd;
import pandas_datareader as pdr;
import datetime;

C:\ProgramData\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Carregar variaveis que serão usadas por várias partes do código
vHoje = datetime.date.today()
vDataFim = vHoje
vDataIni = vDataFim.replace(year = (vHoje.year-1) )
print(vDataIni,"|",vDataFim)

2019-09-23 | 2020-09-23


## Método 1 - Planilha do Google & Python

Primeiramente crie uma pasta no seu Google Drive, para salvar as planilhas que você gerará, que receberão os dados do Google Finance.

As formulas a usar nas planilhas são bem simples e intuitivas. Em caso de duvidas, basta acessar a [documentação](https://https://support.google.com/docs/answer/3093281?hl=pt-BR).

Eu criei uma planilha, buscando os dados da cotação do dolar (em uma aba) e as cotações das ações do Magazine Luiza (em outra aba) dos ultimo ano.

Depois de criar essas formulas, cliquei em compartilhar, e copiei o link, e agora só trabalhar com a planilha aqui no Python

Vamos alterar o sufixo do arquivo, para poder obter o arquivo passivel de leitura:

In [3]:
# Link de compartilhamento da minha planilha criada
# você tem acesso a essa planilha com esse link, pois é um arquivo meu, compartilhado
planilha = 'https://docs.google.com/spreadsheets/d/14myIkRTo_EtWu11j_h8ZczV1Bexo3cC-ObEJynKmVms/edit?usp=sharing'

# Alterando o sufixo do link, para buscar um arquivo com leitura direta
planilha = planilha.replace('/edit?usp=sharing', '/export?format=xlsx')
planilha

'https://docs.google.com/spreadsheets/d/14myIkRTo_EtWu11j_h8ZczV1Bexo3cC-ObEJynKmVms/export?format=xlsx'

In [4]:
# Cotação do Dolar
dolar = pd.read_excel(planilha,sheet_name='Dolar')

# Organizei por data, para que a mais recente ficasse primeiro
dolar = dolar.sort_values(by = ['Date'], ascending=False)

# mostrando dataset
dolar

,Date,Close
363,2020-09-22 23:57:59.904,5.47140
362,2020-09-21 23:57:59.904,5.41410
361,2020-09-20 23:57:59.904,5.39130
360,2020-09-19 23:57:59.904,5.39130
359,2020-09-18 23:57:59.904,5.39130
...,...,...
4,2019-09-28 23:57:59.904,4.17005
3,2019-09-27 23:57:59.904,4.15870
2,2019-09-26 23:57:59.904,4.17030
1,2019-09-25 23:57:59.904,4.14860


Com esse método, a planilha "se atualiza" sozinha, ou seja, de você invocar o link, ele vai fazer o download do arquivo, o qual tem em sua formula um "TODAY()" como parametro de data fim, logo, os dados serão atualizados até o mais recente possivel.

Vamos ver a planilha de ações?
Olhe só...

In [5]:
acao_no_google = pd.read_excel(planilha,sheet_name='Magalu')
acao_no_google = acao_no_google.sort_values(by = ['Date'], ascending=False)
acao_no_google.head()

,Date,Close
246,2020-09-22 16:56:00.384,89.49
245,2020-09-21 16:56:00.384,88.70
244,2020-09-18 16:56:00.384,87.16
243,2020-09-17 16:56:00.384,87.10
242,2020-09-16 16:56:00.384,89.17


**Conclusão do método**
Você pode criar uma planilha, buscando os dados que bem quer, usando a comodidade das formulas do Google Sheets (que são muito faceis), e depois só chamar aqui no Python, sem a necessidade de inumeras linhas de código.

## Método 2 - Dados disponibilizados pelo Governo
**SOMENTE DOLAR**

O governo brasileiro disponibiliza vários dados abertos para analise, os quais podemos citar os indicadores financeiros, a cotação do Dolar, entre outros.

Para esse método, vamos buscar a url que retorna as informações necessárias, e vamos invocá-la em nosso projeto.

Acesse [aqui](https://http://dados.gov.br/dataset/dolar-americano-usd-todos-os-boletins-diarios) a documentação da API.

In [6]:
# link API disponibilizada pelo governo
link_gov = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01%2F01%2F1900'&@dataFinalCotacao='09%2F20%2F2020'&$top=100&$format=text/csv&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"

Vimos aqui que no meio da URL temos parametros, que são usados como filtro na busca, logo, identificamos o padrão da data, e vamos "converter" nossa data para esse padrão e rfemontar o link com nossas datas definidas

In [7]:
# função para tratar a data no formato da API
def trata_data(x):
  m = str(x.month).zfill(2)
  d = str(x.day).zfill(2)
  y = m + '%2F' + d + '%2F' + str(x.year)
  return y

In [8]:
# dicionario com as datas a alterar
datas_alterar = {
    '01%2F01%2F1900' : trata_data(vDataIni),
    '09%2F20%2F2020' : trata_data(vDataFim)
}

In [9]:
# para cada item do dicionario, veja se tem correspondencia no texto...se sim, altere
for key in datas_alterar.keys():
    link_gov = link_gov.replace(key, datas_alterar[key])

In [10]:
# importando DATASET do governo
df_gov = pd.read_csv(link_gov, sep = ',')

# alterando as colunas numerais que estão com ',' no lugar do '.' (padrão Python)
df_gov['cotacaoCompra'] = df_gov['cotacaoCompra'].apply(lambda x: x.replace(',', '.'))
df_gov['cotacaoVenda'] = df_gov['cotacaoVenda'].apply(lambda x: x.replace(',', '.'))
df_gov

,cotacaoCompra,cotacaoVenda,dataHoraCotacao
0,4.1728,4.1734,2019-09-23 13:11:42.21
1,4.1715,4.1721,2019-09-24 13:07:08.372
2,4.1821,4.1827,2019-09-25 13:10:01.22
3,4.1463,4.1469,2019-09-26 13:09:34.075
4,4.1581,4.1587,2019-09-27 13:08:51.782
...,...,...,...
95,4.2471,4.2477,2020-02-06 13:05:35.389
96,4.307,4.3076,2020-02-07 13:10:22.152
97,4.3189,4.3195,2020-02-10 13:11:54.282
98,4.3142,4.3148,2020-02-11 13:05:32.491


**Conclusão do método**
Aqui não precisamos criar nenhum "trampolim" fora do Python, e tivemos que trabalhar um pouco mais em código.

## Método 3 - Yahoo Finance com script direto

Mas você pode fazer pusando o "pandas_datareader" que retornará dados com base no Yahoo Finance.

A diferença, é que você faz tudo no Python, sem necessidade de uma planilha fora.

A documentação da API está disponivel [aqui](https://https://pandas-datareader.readthedocs.io/en/latest/).

In [11]:
# buscar o valor das ações do Magazine Luiza
acao_no_yahoo = pdr.get_data_yahoo('MGLU3.SA', start=vDataIni, end=vDataFim)
acao_no_yahoo

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-09-23,36.580002,35.740002,36.580002,35.990002,6168400.0,35.658260
2019-09-24,37.000000,35.970001,36.099998,36.770000,9428700.0,36.431068
2019-09-25,36.869999,35.750000,36.549999,36.720001,5665200.0,36.381531
2019-09-26,37.270000,36.509998,36.810001,36.570000,6935100.0,36.232910
2019-09-27,36.840000,36.150002,36.560001,36.799999,4086600.0,36.460789
...,...,...,...,...,...,...
2020-09-16,90.900002,88.629997,89.820000,89.169998,9244900.0,89.169998
2020-09-17,89.300003,86.660004,88.589996,87.099998,12386700.0,87.099998
2020-09-18,89.550003,86.040001,87.599998,87.160004,15117100.0,87.160004


**Conclusão do método**
Aqui não precisamos criar nenhum "trampolim" fora do Python, e tivemos que trabalhar um pouco mais em código.
Cabe verificar se as opções na captura (get) de dados dessa forma tem mais possibilidades (mais parametros) que o Google Finance, e decidir o que é melhor e mais usual para seu dia a dia.

## **CONCLUSÃO**


Usando o método com auxilio da planilha do GOOGLE, sua consulta fica mais "ALTO NIVEL", ou seja, você consegue usar os exemplos prontos do Google Sheets, e agilizar o trabalho, mas, se você quer algo direto ao ponto, o método YAHOO (ou API do governo - para a cotação do Dolar) parece mais interessante.

Vale você testar os parametros disponiveis no Google Sheets pra ver se consegue obter mais (ou menos) opções que o Yahoo, dai tomar uma decisão.

Bons testes a você!

### Conheça meu trabalho

<a href="https://www.joviano.com">
<img src="https://raw.githubusercontent.com/jovianosilveira/img/master/joviano-silveira-site-logo.png" alt="joviano.com" width="5%">
</a>

<a href="https://www.youtube.com/jovianosilveira">
<img src="https://github.com/jovianosilveira/img/blob/master/joviano-silveira-youtube-logo.png?raw=true" alt="joviano.com" width="5%">
</a>

<a href="http://www.linkedin.com/in/jovianosilveira">
<img src="https://github.com/jovianosilveira/img/blob/master/joviano-silveira-linkedin-logo.png?raw=true" alt="joviano.com" width="5%">
</a>

<a href="https://www.instagram.com/jovi.silveira">
<img src="https://github.com/jovianosilveira/img/blob/master/joviano-silveira-instagram-logo.png?raw=true" alt="joviano.com" width="5%">
</a>

<a href="https://www.facebook.com/jovi.silveira.cursos">
<img src="https://github.com/jovianosilveira/img/blob/master/joviano-silveira-facebook-logo.png?raw=true" alt="joviano.com" width="5%">
</a>

### Conheça os cursos que fiz para você

<a href="https://www.joviano.com/cursos-para-voce">
<img src="https://github.com/jovianosilveira/img/blob/master/joviano-silveira-cursos-logo.png?raw=true" alt="joviano.com" width="10%">
</a>